In [2]:
username = "alextran21211"
password = "database12345"
host = "cluster0.zsych.mongodb.net"
database = "group_5_project"
collection = "co2_emission"


In [3]:
from pymongo import MongoClient # import mongo client to connect
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, isnan, mean, stddev, min, max, desc
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import urllib.parse
from sklearn.preprocessing import MinMaxScaler

In [4]:
username = username
password = urllib.parse.quote(password)
host = host
url = "mongodb+srv://{}:{}@{}/?retryWrites=true&w=majority".format(username,
password, host)
# connect to the database
client = MongoClient(url)

In [5]:
# retrive more features for analyzing
db = client['group_5_project']
collection = db["co2_emission"]

documents = collection.find()

data = []

for doc in documents:
    for country, country_data in doc.items():
        if country == "_id":
            continue
        iso_code = country_data.get('iso_code')
        for entry in country_data.get('data', []):
            year = entry.get('year')
            population = entry.get('population')
            co2 = entry.get('cumulative_luc_co2')
            coal_co2 = entry.get('coal_co2')
            oil_co2 = entry.get('oil_co2')
            gas_co2 = entry.get('gas_co2')
            cement_co2 = entry.get('cement_co2')
            flaring_co2 = entry.get('flaring_co2')
            other_industry_co2 = entry.get('other_industry_co2')
            
            data.append({
                'Country': country,
                'ISO_Code': iso_code,
                'Year': year,
                'Population': population,
                'CO2': co2,
                'Coal_CO2': coal_co2,
                'Oil_CO2': oil_co2,
                'Gas_CO2': gas_co2,
                'Cement_CO2': cement_co2,
                'Flaring_CO2': flaring_co2,
                'Other_Industry_CO2': other_industry_co2
            })

df = pd.DataFrame(data)


In [6]:
print(df.head())

       Country ISO_Code  Year  Population        CO2  Coal_CO2  Oil_CO2  \
0  Afghanistan      AFG  1850   3752993.0   2.979601       NaN      NaN   
1  Afghanistan      AFG  1851   3767956.0   5.981443       NaN      NaN   
2  Afghanistan      AFG  1852   3783940.0   9.002998       NaN      NaN   
3  Afghanistan      AFG  1853   3800954.0  12.041333       NaN      NaN   
4  Afghanistan      AFG  1854   3818038.0  15.094068       NaN      NaN   

   Gas_CO2  Cement_CO2  Flaring_CO2  Other_Industry_CO2  
0      NaN         NaN          NaN                 NaN  
1      NaN         NaN          NaN                 NaN  
2      NaN         NaN          NaN                 NaN  
3      NaN         NaN          NaN                 NaN  
4      NaN         NaN          NaN                 NaN  


In [7]:
print(df.tail())

        Country ISO_Code  Year  Population          CO2  Coal_CO2   Oil_CO2  \
47410  Zimbabwe      ZWE  2018  15052191.0  2269.845703  6.096896  4.059712   
47411  Zimbabwe      ZWE  2019  15354606.0  2277.924316  5.571349  3.731338   
47412  Zimbabwe      ZWE  2020  15669663.0  2285.533447  4.308864  3.044784   
47413  Zimbabwe      ZWE  2021  15993525.0  2292.650146  4.548245  3.317047   
47414  Zimbabwe      ZWE  2022  16320539.0  2299.374268  5.027008  3.298107   

       Gas_CO2  Cement_CO2  Flaring_CO2  Other_Industry_CO2  
47410      0.0    0.557990          0.0                 NaN  
47411      0.0    0.472742          0.0                 NaN  
47412      0.0    0.495991          0.0                 NaN  
47413      0.0    0.530865          0.0                 NaN  
47414      0.0    0.530865          0.0                 NaN  


In [8]:
print(df.info())  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47415 entries, 0 to 47414
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             47415 non-null  object 
 1   ISO_Code            39548 non-null  object 
 2   Year                47415 non-null  int64  
 3   Population          39414 non-null  float64
 4   CO2                 37022 non-null  float64
 5   Coal_CO2            25075 non-null  float64
 6   Oil_CO2             25111 non-null  float64
 7   Gas_CO2             25083 non-null  float64
 8   Cement_CO2          23764 non-null  float64
 9   Flaring_CO2         24992 non-null  float64
 10  Other_Industry_CO2  2593 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 4.0+ MB
None


In [9]:
print(df.isnull().sum()) 

Country                   0
ISO_Code               7867
Year                      0
Population             8001
CO2                   10393
Coal_CO2              22340
Oil_CO2               22304
Gas_CO2               22332
Cement_CO2            23651
Flaring_CO2           22423
Other_Industry_CO2    44822
dtype: int64


In [10]:
df['Population'] = df['Population'].fillna(df['Population'].mean()) 
df['CO2'] = df['CO2'].fillna(df['CO2'].mean())  
df['ISO_Code'] = df['ISO_Code'].dropna

In [11]:
df['CO2_per_capita'] = df['CO2'] / df['Population']

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
print(df.isnull().sum()) 

Country                   0
ISO_Code                  0
Year                      0
Population                0
CO2                       0
Coal_CO2              22340
Oil_CO2               22304
Gas_CO2               22332
Cement_CO2            23651
Flaring_CO2           22423
Other_Industry_CO2    44822
CO2_per_capita            0
dtype: int64


In [14]:
df = df.drop(columns=['ISO_Code'])
print(df.head())

       Country  Year  Population        CO2  Coal_CO2  Oil_CO2  Gas_CO2  \
0  Afghanistan  1850   3752993.0   2.979601       NaN      NaN      NaN   
1  Afghanistan  1851   3767956.0   5.981443       NaN      NaN      NaN   
2  Afghanistan  1852   3783940.0   9.002998       NaN      NaN      NaN   
3  Afghanistan  1853   3800954.0  12.041333       NaN      NaN      NaN   
4  Afghanistan  1854   3818038.0  15.094068       NaN      NaN      NaN   

   Cement_CO2  Flaring_CO2  Other_Industry_CO2  CO2_per_capita  
0         NaN          NaN                 NaN    7.939267e-07  
1         NaN          NaN                 NaN    1.587450e-06  
2         NaN          NaN                 NaN    2.379266e-06  
3         NaN          NaN                 NaN    3.167977e-06  
4         NaN          NaN                 NaN    3.953357e-06  


In [15]:
scaler = MinMaxScaler()
df[['Population', 'CO2', 'CO2_per_capita']] = scaler.fit_transform(df[['Population', 'CO2', 'CO2_per_capita']])

In [16]:
print(df.describe())        # Summary statistics
print(df.head())

               Year    Population           CO2      Coal_CO2       Oil_CO2  \
count  47415.000000  47415.000000  47415.000000  25075.000000  25111.000000   
mean    1926.781609      0.007643      0.017994    156.963928    112.920677   
std       59.561600      0.037602      0.048560    767.115976    630.125684   
min     1750.000000      0.000000      0.000000      0.000000      0.000000   
25%     1883.000000      0.000073      0.005530      0.000000      0.084272   
50%     1930.000000      0.000521      0.006442      0.703386      1.322799   
75%     1976.000000      0.007104      0.017994     16.962487     13.868931   
max     2022.000000      1.000000      1.000000  15219.303711  12377.949219   

            Gas_CO2    Cement_CO2   Flaring_CO2  Other_Industry_CO2  \
count  25083.000000  23764.000000  24992.000000         2593.000000   
mean      51.278100      9.109399      3.636449           17.499371   
std      316.287376     66.662220     20.916841           41.335942   
min 

In [17]:
# Example list of countries (you can expand this list as needed)
countries_list = [
    'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 
    'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 
    'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 
    'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba', 
    'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei', 
    'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 
    'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Colombia', 
    'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 
    'Curacao', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 
    'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 
    'Ethiopia', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 
    'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong', 
    'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 
    'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', 'Kosovo', 'Kuwait', 
    'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 
    'Lithuania', 'Luxembourg', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 
    'Mauritania', 'Mauritius', 'Mexico', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 
    'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Zealand', 'Nicaragua', 'Niger', 
    'Nigeria', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Panama', 'Papua New Guinea', 'Paraguay', 
    'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Russia', 'Rwanda', 'Saint Lucia', 
    'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 
    'Senegal', 'Serbia', 'Seychelles', 'Singapore', 'Slovakia', 'Slovenia', 'Solomon Islands', 
    'Somalia', 'South Africa', 'South Korea', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'Sweden', 
    'Switzerland', 'Syria', 'Taiwan', 'Tajikistan', 'Tanzania', 'Thailand', 'Togo', 'Trinidad and Tobago', 
    'Tunisia', 'Turkey', 'Turkmenistan', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 
    'United States', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Vatican', 'Venezuela', 'Vietnam', 'Yemen', 
    'Zambia', 'Zimbabwe'
]

# Filter the DataFrame to keep only rows with countries in the countries_list 
df = df[(df['Country'].isin(countries_list)) & (df['Year'] > 1950)]




In [18]:
df.to_csv("../data/processed/co2_emission_preprocessed.csv", index=False)


In [19]:
df.head()

,Country,Year,Population,CO2,Coal_CO2,Oil_CO2,Gas_CO2,Cement_CO2,Flaring_CO2,Other_Industry_CO2,CO2_per_capita
101,Afghanistan,1951,0.000949,0.006129,0.025648,0.065952,0.0,0.0,0.0,NaN,0.000007
102,Afghanistan,1952,0.000961,0.006140,0.031708,0.059892,0.0,0.0,0.0,NaN,0.000007
103,Afghanistan,1953,0.000974,0.006153,0.037949,0.068307,0.0,0.0,0.0,NaN,0.000007
104,Afghanistan,1954,0.000986,0.006167,0.042502,0.063754,0.0,0.0,0.0,NaN,0.000007
105,Afghanistan,1955,0.001000,0.006181,0.062288,0.091600,0.0,0.0,0.0,NaN,0.000007
